In [4]:
import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime
import random

In [13]:
# connect to the database
con = sqlite3.connect("D:/Programs/SQLiteStudio/club_events")

# New Member Registration

In [6]:
first_name = 'Johndale Alfred'
last_name = 'Julian'
middle_name = 'Andres'
email = 'johndalealfred.julian@benilde.edu.ph'
birthday = '1995-02-26'
salutation = 'Mr.'
gender = 'Male'

street_address = '4809 Lena Lane'
city = 'Polkville'
province = 'Mississippi'
country = 'United States'
zip_code = '39117'

In [14]:
def register_address():
    address_tuple = (street_address, city, province, country, zip_code)
    cursor = con.cursor()
    sql = '''INSERT INTO ADDRESSES (street_address, city, province, country, zip) 
            VALUES (?, ?, ?, ?, ?)'''
    cursor.execute(sql, address_tuple)
    address_id = cursor.lastrowid
    con.commit()
    return address_id
    

def register_member():
    member_tuple = (first_name, last_name, middle_name, email, birthday, salutation, gender, register_address())
    cursor = con.cursor()
    
    sql = '''INSERT INTO MEMBERS (first_name, last_name, middle_name, email, birthday, salutation, gender, address_id)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)'''
    
    cursor.execute(sql, member_tuple)
    member_id = cursor.lastrowid
    con.commit()
    
    cursor = con.cursor()
    
    member_details = pd.read_sql('SELECT * FROM MEMBERS WHERE id = {id}'.format(id = member_id), con) 
    print('Successfully registered:')
    con.commit()
    return member_details.style.hide_index()

In [15]:
register_member()

Successfully registered:


id,first_name,last_name,middle_name,email,birthday,salutation,gender,address_id
12,Johndale Alfred,Julian,Andres,johndalealfred.julian@benilde.edu.ph,1995-02-26,Mr.,Male,12


# Create Dinner Event

In [17]:
def create_dinner(desc, datetime):
    dinner = (desc, datetime) #convert to tuple
    sql ='''INSERT INTO DINNERS (description, datetime) VALUES (?, ?)'''
    
    cursor = con.cursor()
    cursor.execute(sql, dinner)
    
    dinner_id = cursor.lastrowid
    
    dinner_details = pd.read_sql('SELECT * FROM DINNERS WHERE id = {id}'.format(id = dinner_id), con)
    
    #send_invites(dinner_id)
    
    con.commit()
    print('Successfully created dinner event:')
    print(dinner_details)
    print()

In [18]:
create_dinner("Sample Dinner", "2021-09-13")

Successfully created dinner event:
   id    description    datetime
0  11  Sample Dinner  2021-09-13



# Send Dinner Invites to All Club Members

In [19]:
def get_members_df():
    members_sql = '''SELECT * FROM MEMBERS'''
    cursor = con.cursor()
    
    members_df = pd.read_sql_query(members_sql, con)
    return members_df
    
def send_invites(dinner_id):
    members_df = get_members_df()
    
    cursor = con.cursor()
    sql = '''INSERT INTO INVITATIONS (dinner_id, member_id, invite_date) 
            VALUES (?, ?, ? )'''
    
    for x in [item for item in members_df['id']]:
        invite_tuple = (dinner_id, x, str(datetime.now()).split('.')[0])
        cursor.execute(sql, invite_tuple)
        
    con.commit()
    print('Successfully sent invites for event id {id}!'.format(id = dinner_id))
        

In [21]:
send_invites(dinner_id = 11)

Successfully sent invites for event id 11!


# Show Mains, Sides, and Desserts

In [22]:
def get_main_dishes():
    sql = '''SELECT * FROM MAIN_DISHES'''
    cursor = con.cursor()
    
    df = pd.read_sql_query(sql, con)
    return df

def get_side_dishes():
    sql = '''SELECT * FROM SIDE_DISHES'''
    cursor = con.cursor()
    
    df = pd.read_sql_query(sql, con)
    return df

def get_desserts():
    sql = '''SELECT * FROM DESSERTS'''
    cursor = con.cursor()
    
    df = pd.read_sql_query(sql, con)
    return df

dishes = pd.concat([get_main_dishes(), get_side_dishes(), get_desserts()], keys = ['Mains', 'Sides', 'Desserts'])
dishes

id                          description
Mains    0   1                       Butter Chicken
         1   2                         Palak Paneer
         2   3                     Grilled Chicken 
         3   4      Bombay Grilled Chutney Sandwich
         4   5               Persian Chicken Kebabs
         5   6                  Spicy Pork Vindaloo
         6   7                      Beef Enchiladas
         7   8                       Coconut Shrimp
         8   9                General Tso's Chicken
         9  10  Greek Lamb Gyros with Taztiki Sauce
Sides    0   1                       Roast Potatoes
         1   2                      Creamed Spinach
         2   3              Mashed Butternut Squash
         3   4         Green Bean Casserole Bundles
         4   5                  Bacon Avocado Fries
         5   6                             Coleslaw
         6   7                Cauliflower Au Gratin
         7   8                Brussels Sprout Chips
         8   9                      Mashed Potatoes
         9  10                    Grilled Asparagus
Desserts 0   1                             Tiramisu
         1   2                              Churros
         2   3                              S'mores
         3   4                              Baklava
         4   5                             Rum Cake
         5   6                        Malva Pudding
         6   7                          Marble Cake
         7   8                          Pandan Cake
         8   9                           Pound Cake
         9  10                       Ice Cream Cake

# RSVP Yes, Confirm Food

In [25]:
def accept_invite(member_id, dinner_id, main_dish, side_dish_1, side_dish_2, dessert):
    cursor = con.cursor()
    sql = '''UPDATE INVITATIONS SET main_dish_id = ?,
                                    side_dish_1_id = ?,
                                    side_dish_2_id = ?,
                                    dessert_id = ?,
                                    rsvp = 1,
                                    response_date = ?
                WHERE member_id = ? AND dinner_id = ? '''
    tpl = (main_dish, side_dish_1, side_dish_2, dessert, member_id, dinner_id, str(datetime.now()).split('.')[0])

    cursor.execute(sql, tpl)
    con.commit()

In [24]:
accept_invite(member_id = 12, 
              dinner_id = 11, 
              main_dish = random.randrange(1, 11), 
              side_dish_1 = random.randrange(1, 11), 
              side_dish_2 = random.randrange(1, 11),
             dessert = random.randrange(1, 11))

# RSVP No

In [26]:
def reject_invite(member_id, dinner_id):
    cursor = con.cursor()
    sql = '''UPDATE INVITATIONS SET rsvp = false,
                                    response_date = ? 
                    WHERE member_id = ? AND dinner_id = ? '''
    tpl = (str(datetime.now()).split('.')[0], member_id, dinner_id)
    cursor.execute(sql, tpl)
    con.commit()
    print('Rejected invite for member id {mid} for dinner id {did}'.format(mid = member_id, did = dinner_id))

In [27]:
reject_invite(member_id = 1, dinner_id = 11)

Rejected invite for member id 1 for dinner id 11


# Food Preparation for Dinner Event

In [29]:
dinner_id = 1
cursor = con.cursor()

def get_food_prep():
    df1 = pd.read_sql_query('''SELECT * FROM INVITATIONS''', con)
    df2 = df1[df1['dinner_id'] == dinner_id]

    df2.groupby(['main_dish_id'])['id'].count()

    x = df2.groupby(['side_dish_1_id'], as_index=False)['id'].count()
    y = df2.groupby(['side_dish_2_id'], as_index=False)['id'].count()
    x.rename(columns = {'side_dish_1_id' : 'id', 'id' : 'count'}, inplace=True)
    y.rename(columns = {'side_dish_2_id' : 'id', 'id' : 'count'}, inplace=True)
    df = x.append(y)
    df2 = df.groupby('id').sum()

    df3 = pd.read_sql_query('''SELECT * FROM SIDE_DISHES''', con)
    df4 = df3.merge(df2, on = 'id')
    sides = df4[['description', 'count']].sort_values('count', ascending = False)

    df2 = df1[df1['dinner_id'] == 1]
    df3 = df2.groupby(['main_dish_id'], as_index=False)['id'].count()
    df3.rename(columns = {'main_dish_id' : 'id', 'id' :'count'}, inplace=True)
    df4 = pd.read_sql_query('''SELECT * FROM MAIN_DISHES''', con)
    df4 = df4.merge(df3, on ='id')
    mains = df4[['description', 'count']].sort_values('count', ascending = False)

    df2 = df1[df1['dinner_id'] == dinner_id]
    df3 = df2.groupby(['dessert_id'], as_index = False)['id'].count()
    df3.rename(columns = {'dessert_id' : 'id', 'id' : 'count'}, inplace = True)
    df4 = pd.read_sql_query('''SELECT * FROM DESSERTS''', con)
    df4 = df4.merge(df3, on = 'id')
    desserts = df4[['description', 'count']].sort_values('count', ascending = False)
    desserts
    df = pd.concat([mains, sides, desserts], keys = ['Mains', 'Sides', 'Desserts'])
    return df

get_food_prep()

description  count
Mains    0                   Butter Chicken      3
         2                 Grilled Chicken       3
         1                     Palak Paneer      2
         3  Bombay Grilled Chutney Sandwich      2
Sides    3     Green Bean Casserole Bundles      5
         0                   Roast Potatoes      3
         1                  Creamed Spinach      2
         2          Mashed Butternut Squash      2
         5                         Coleslaw      2
         7                  Mashed Potatoes      2
         8                Grilled Asparagus      2
         4              Bacon Avocado Fries      1
         6            Cauliflower Au Gratin      1
Desserts 1                          Churros      2
         3                    Malva Pudding      2
         4                      Marble Cake      2
         5                       Pound Cake      2
         0                         Tiramisu      1
         2                         Rum Cake      1

# Register Guest Attendance

In [31]:
member_id = 12
dinner_id = 11

def attend():
    cursor = con.cursor()
    sql = '''UPDATE INVITATIONS SET
                dinner_attended = 1
            WHERE member_id = ? AND dinner_id = ? '''
    tpl = member_id, dinner_id
    cursor.execute(sql, tpl)
    
    print('Confirmed attendance of member {mid} to dinner {did}'.format(mid = member_id, did = dinner_id))
    con.commit()
    
attend()

Confirmed attendance of member 12 to dinner 11
